# Notebook de inferencia de ventas 2025
En este notebook se importan las librerías necesarias y se carga el archivo de ventas para realizar análisis y predicciones.

In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import streamlit as st
import holidays

In [2]:
# Cargar archivo de ventas_2025_Inferencia en inferencia_df
inferencia_path = '../data/raw/inferencia/ventas_2025_inferencia.csv'
inferencia_df = pd.read_csv(inferencia_path)

In [ ]:
# Mostrar las primeras filas de inferencia_df para verificar la carga
display(inferencia_df.head())

,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,Amazon,Decathlon,Deporvillage
0,2025-10-25,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,26.0,113.13,2941.38,89.51,113.43,104.78
1,2025-10-25,PROD_002,Adidas Ultraboost 23,Running,Zapatillas Running,135,True,27.0,141.89,3831.03,128.73,112.91,122.88
2,2025-10-25,PROD_003,Asics Gel Nimbus 25,Running,Zapatillas Running,85,False,5.0,85.79,428.95,84.28,74.51,85.57
3,2025-10-25,PROD_004,New Balance Fresh Foam X 1080v12,Running,Zapatillas Running,75,False,3.0,76.19,228.57,75.54,70.32,71.13
4,2025-10-25,PROD_005,Nike Dri-FIT Miler,Running,Ropa Running,35,False,3.0,35.48,106.44,33.84,31.32,34.41


In [4]:
# ...código existente...

# 1. Convertir columna fecha a datetime
inferencia_df['fecha'] = pd.to_datetime(inferencia_df['fecha'])

# 2. Variables temporales
inferencia_df['año'] = inferencia_df['fecha'].dt.year
inferencia_df['mes'] = inferencia_df['fecha'].dt.month
inferencia_df['dia_mes'] = inferencia_df['fecha'].dt.day
inferencia_df['dia_semana'] = inferencia_df['fecha'].dt.dayofweek
inferencia_df['nombre_dia'] = inferencia_df['fecha'].dt.day_name()
inferencia_df['trimestre'] = inferencia_df['fecha'].dt.quarter
inferencia_df['semana_año'] = inferencia_df['fecha'].dt.isocalendar().week

# 3. Festivos España
es_festivos = inferencia_df['fecha'].isin([d for d in holidays.country_holidays('ES', years=inferencia_df['año'].unique())])
inferencia_df['es_festivo'] = es_festivos

# 4. Fin de semana
inferencia_df['es_fin_semana'] = inferencia_df['dia_semana'].isin([5, 6])

# 5. Black Friday (último viernes de noviembre)
def es_black_friday(fecha):
    if fecha.month == 11:
        fridays = pd.date_range(start=f'{fecha.year}-11-01', end=f'{fecha.year}-11-30', freq='W-FRI')
        return fecha in fridays[-1:]
    return False
inferencia_df['es_BlackFriday'] = inferencia_df['fecha'].apply(es_black_friday)

# 6. Cyber Monday (primer lunes después de Black Friday)
def es_cyber_monday(fecha):
    if fecha.month == 11 or fecha.month == 12:
        fridays = pd.date_range(start=f'{fecha.year}-11-01', end=f'{fecha.year}-11-30', freq='W-FRI')
        if len(fridays) > 0:
            cyber_monday = fridays[-1] + pd.Timedelta(days=3)
            return fecha == cyber_monday
    return False
inferencia_df['es_cyber_monday'] = inferencia_df['fecha'].apply(es_cyber_monday)

# 7. Día laborable
inferencia_df['es_laborable'] = (~inferencia_df['es_fin_semana']) & (~inferencia_df['es_festivo'])

# 8. Inicio/fin de mes
inferencia_df['es_inicio_mes'] = inferencia_df['dia_mes'] <= 3
inferencia_df['es_fin_mes'] = inferencia_df['dia_mes'] >= (inferencia_df['fecha'] + pd.offsets.MonthEnd(0)).dt.day - 2

# 9. Descuento porcentaje
inferencia_df['descuento_porcentaje'] = ((inferencia_df['precio_venta'] - inferencia_df['precio_base']) / inferencia_df['precio_base']) * 100

# 10. Precio competencia y ratio_precio
competidores = ['Amazon', 'Decathlon', 'Deporvillage']
inferencia_df['precio_competencia'] = inferencia_df[competidores].mean(axis=1)
inferencia_df['ratio_precio'] = inferencia_df['precio_venta'] / inferencia_df['precio_competencia']

# 11. Variables de lag y media móvil (por producto)
lags = [1,2,3,4,5,6,7]
for lag in lags:
    inferencia_df[f'unidades_vendidas_lag_{lag}'] = inferencia_df.groupby('producto_id')['unidades_vendidas'].shift(lag)
inferencia_df['unidades_vendidas_ma_7'] = inferencia_df.groupby('producto_id')['unidades_vendidas'].rolling(window=7, min_periods=1).mean().reset_index(level=0, drop=True)

# 12. Codificación one-hot de nombre, categoria, subcategoria (con sufijo _h)
for col in ['nombre', 'categoria', 'subcategoria']:
    inferencia_df[f'{col}_h'] = inferencia_df[col]
cols_h = ['nombre_h', 'categoria_h', 'subcategoria_h']
inferencia_df = pd.get_dummies(inferencia_df, columns=cols_h, prefix=cols_h)

# 13. Eliminar registros de octubre y dejar solo noviembre
inferencia_df = inferencia_df[inferencia_df['mes'] == 11].copy()

# 14. Guardar el DataFrame transformado
inferencia_df.to_csv('../data/processed/inferencia_df_transformado.csv', index=False)

C:\Users\white\AppData\Local\Temp\ipykernel_8744\4250001167.py:16: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  es_festivos = inferencia_df['fecha'].isin([d for d in holidays.country_holidays('ES', years=inferencia_df['año'].unique())])


In [5]:
inferencia_df

,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,...,subcategoria_h_Esterilla Yoga,subcategoria_h_Mancuernas Ajustables,subcategoria_h_Mochila Trekking,subcategoria_h_Pesa Rusa,subcategoria_h_Pesas Casa,subcategoria_h_Rodillera Yoga,subcategoria_h_Ropa Montaña,subcategoria_h_Ropa Running,subcategoria_h_Zapatillas Running,subcategoria_h_Zapatillas Trail
168,2025-11-01,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,NaN,115.00,NaN,...,False,False,False,False,False,False,False,False,True,False
169,2025-11-01,PROD_002,Adidas Ultraboost 23,Running,Zapatillas Running,135,True,NaN,135.00,NaN,...,False,False,False,False,False,False,False,False,True,False
170,2025-11-01,PROD_003,Asics Gel Nimbus 25,Running,Zapatillas Running,85,False,NaN,86.39,NaN,...,False,False,False,False,False,False,False,False,True,False
171,2025-11-01,PROD_004,New Balance Fresh Foam X 1080v12,Running,Zapatillas Running,75,False,NaN,74.09,NaN,...,False,False,False,False,False,False,False,False,True,False
172,2025-11-01,PROD_005,Nike Dri-FIT Miler,Running,Ropa Running,35,False,NaN,34.76,NaN,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883,2025-11-30,PROD_020,Quechua MH500,Outdoor,Ropa Montaña,80,False,NaN,79.64,NaN,...,False,False,False,False,False,False,True,False,False,False
884,2025-11-30,PROD_021,Manduka PRO Yoga Mat,Wellness,Esterilla Yoga,130,True,NaN,130.00,NaN,...,True,False,False,False,False,False,False,False,False,False
885,2025-11-30,PROD_022,Gaiam Premium Yoga Block,Wellness,Bloque Yoga,20,False,NaN,20.18,NaN,...,False,False,False,False,False,False,False,False,False,False
886,2025-11-30,PROD_023,Liforme Yoga Pad,Wellness,Rodillera Yoga,35,False,NaN,34.79,NaN,...,False,False,False,False,False,True,False,False,False,False


In [6]:
inferencia_df.shape

(720, 82)

In [7]:
inferencia_df.columns

Index(['fecha', 'producto_id', 'nombre', 'categoria', 'subcategoria',
       'precio_base', 'es_estrella', 'unidades_vendidas', 'precio_venta',
       'ingresos', 'Amazon', 'Decathlon', 'Deporvillage', 'año', 'mes',
       'dia_mes', 'dia_semana', 'nombre_dia', 'trimestre', 'semana_año',
       'es_festivo', 'es_fin_semana', 'es_BlackFriday', 'es_cyber_monday',
       'es_laborable', 'es_inicio_mes', 'es_fin_mes', 'descuento_porcentaje',
       'precio_competencia', 'ratio_precio', 'unidades_vendidas_lag_1',
       'unidades_vendidas_lag_2', 'unidades_vendidas_lag_3',
       'unidades_vendidas_lag_4', 'unidades_vendidas_lag_5',
       'unidades_vendidas_lag_6', 'unidades_vendidas_lag_7',
       'unidades_vendidas_ma_7', 'nombre_h_Adidas Own The Run Jacket',
       'nombre_h_Adidas Ultraboost 23', 'nombre_h_Asics Gel Nimbus 25',
       'nombre_h_Bowflex SelectTech 552', 'nombre_h_Columbia Silver Ridge',
       'nombre_h_Decathlon Bandas Elásticas Set', 'nombre_h_Domyos BM900',
       'n